In [1]:
from src.data_loader.data_loader import BooksDataset
from src.models.mm_model import MmModel
from src.train import Trainer
import torch

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
dataset = BooksDataset(data_dir="../data/books")

In [4]:
model = MmModel(n_users=dataset.n_users,n_items=dataset.n_items,adjacency_matrix=dataset.get_dataset("adjacency_matrix"),embed_size=64,n_layers=3)
model.to(device)

MmModel(
  (E0): Embedding(48752, 64)
  (text_feat): Linear(in_features=48752, out_features=64, bias=True)
  (image_feat): Linear(in_features=48752, out_features=64, bias=True)
)

In [5]:
trainer = Trainer(model=model,dataset=dataset,lr=0.01)

In [6]:
trainer.train(epochs=10,batch_size=1024)

100%|██████████| 3/3 [01:04<00:00, 21.49s/it]


{'precision': array([0.00943811]), 'recall': array([0.01763931]), 'ndcg': array([0.0441644]), 'hit_ratio': array([0.07938412]), 'auc': 0.0}
Epoch 0 Loss 2.078252649307251 Time 78.69586873054504


100%|██████████| 3/3 [01:03<00:00, 21.17s/it]


{'precision': array([0.01017796]), 'recall': array([0.02025845]), 'ndcg': array([0.0478294]), 'hit_ratio': array([0.08618276]), 'auc': 0.0}
Epoch 1 Loss 2.068741194407145 Time 76.09287881851196


100%|██████████| 3/3 [01:02<00:00, 20.82s/it]


...

{'precision': array([0.01113777]), 'recall': array([0.02250734]), 'ndcg': array([0.05030854]), 'hit_ratio': array([0.09318136]), 'auc': 0.0}
Epoch 4 Loss 2.012264021237691 Time 79.52066421508789


100%|██████████| 3/3 [01:06<00:00, 22.24s/it]


{'precision': array([0.01137772]), 'recall': array([0.02253026]), 'ndcg': array([0.05142254]), 'hit_ratio': array([0.09458108]), 'auc': 0.0}
Epoch 5 Loss 1.9809139887491862 Time 80.66909956932068


 87%|████████▋ | 13/15 [00:12<00:01,  1.03it/s]


KeyboardInterrupt: 

In [5]:
device

device(type='cuda')

In [15]:
# create a tenor with 3 items
item = torch.tensor([0,2,3]).to(device)
user = torch.tensor([0,2,3]).to(device)
torch.mm(user,item.view(-1,1))





RuntimeError: self must be a matrix

In [13]:
import numpy as np
def precision_at_k(r, k):
    """Score is precision @ k
    Relevance is binary (nonzero is relevant).
    Returns:
        Precision @ k
    Raises:
        ValueError: len(r) must be >= k
    """
    assert k >= 1
    r = np.asarray(r)[:k]
    return np.mean(r)
# test
r = [0,1,1,0,0,1,0,0,0,1]
precision_at_k(r, 5)

0.4

In [27]:
def recall(rank, ground_truth, N):
    return len(set(rank[:N]) & set(ground_truth)) / float(len(set(ground_truth)))
# test
rank = [0,1,1,0,0,1,0,0,0,1]
ground_truth = [0]
recall(rank, ground_truth, 5)

1.0

In [28]:
set(rank[:5]) & set(ground_truth) # this operation is the intersection of the two sets

{0}

In [9]:
import multiprocessing
multiprocessing.cpu_count()


12

In [2]:
import heapq
Ks = [5]

def rank_list_by_heapq(rating, user_pos_test, test_items):
    item_score = {}
    for i in test_items:
        item_score[i] = rating[i]

    K_max = max(Ks)
    K_max_item_score = heapq.nlargest(K_max, item_score, key=item_score.get)

    r = []
    for i in K_max_item_score:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = 0.
    return r, auc

rating = [0.9,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
user_pos_test = [0,1,2]
test_items = [0,1,2,3,4,5,6,7]
rank_list_by_heapq(rating, user_pos_test, test_items)

([1, 0, 0, 0, 0], 0.0)

In [26]:
item_score = {}
for i in test_items:
    item_score[str(i)] = rating[i]


In [27]:
item_score

{'0': 0.9,
 '1': 0.2,
 '2': 0.3,
 '3': 0.4,
 '4': 0.5,
 '5': 0.6,
 '6': 0.7,
 '7': 0.8}

In [30]:
import pandas as pd

#create data frame with colmuns as items keys and values as scores
df = pd.DataFrame(item_score,index=[0])
df


,0,1,2,3,4,5,6,7
0,0.9,0.2,0.3,0.4,0.5,0.6,0.7,0.8


In [15]:
items

,item,score
0,0,0.9
1,1,0.2
2,2,0.3
3,3,0.4
4,4,0.5
5,5,0.6
6,6,0.7
7,7,0.8


In [32]:
# convert data frame to dict 
items = df.to_dict(orient="records")
items[0]

{'0': 0.9,
 '1': 0.2,
 '2': 0.3,
 '3': 0.4,
 '4': 0.5,
 '5': 0.6,
 '6': 0.7,
 '7': 0.8}